Load Env

In [89]:
from dotenv import load_dotenv

load_dotenv()

True

Get Embeddings

In [90]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=256)

Get Retriever

In [91]:
from langchain_chroma import Chroma
import os 
vector_store = Chroma(
    collection_name="adaptive_rag_test_3",
    embedding_function=embedding,
    chroma_cloud_api_key=os.getenv("CHROMA_API_KEY"),
    tenant=os.getenv("CHROMA_TENANT"),
    database=os.getenv("CHROMA_DATABASE")
)

top_k = 3
retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": top_k},
    )

Define LLM

In [92]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-5-nano", stream_usage=True, temperature=1, reasoning_effort="minimal"
)

Create Contextualize Chain

In [93]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    Runnable,
    RunnablePassthrough,
    RunnableParallel,
    chain,
)
from operator import itemgetter
from pydantic import BaseModel, Field

contextualize_instructions = """Convert the latest user question into a standalone question given the chat history. Don't answer the question, return the question and nothing else (no descriptive text)."""
contextualize_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", contextualize_instructions),
          ("placeholder", "{chat_history}"),
          ("human", "{question}"),
      ]
  )
contextualize_question = contextualize_prompt | llm | StrOutputParser()

@chain
def contextualize_if_needed(input_: dict) -> Runnable:
    if input_.get("chat_history"):
        return contextualize_question
    else:
        return RunnablePassthrough() | itemgetter("question")

Define Query Classifier

In [94]:
from langchain_ollama import ChatOllama

class ClassifierSchema(BaseModel):
    """ Schema for query classification.
    Args:
        classification: One of NOT_RELATED, RELATED
    """
    classification: str = Field(
        ...,
        description="One of NOT_RELATED, RELATED",
    )

classifier_prompt = """You are a query classifier for a question-answering system that uses **Brillar Bank fixed deposit data** as its main source.

Your task is to read a user query and classify it into **one** of the following categories:

- **NOT_RELATED**  
- **RELATED**

---

## Classification Labels

### 1. NOT_RELATED

Use this label if the query **cannot be answered using Brillar Bank fixed deposit data**.

This includes:
- Questions about other banking products (loans, savings accounts, credit cards, etc.)
- Questions about other banks or general finance topics
- Completely unrelated topics (technology, sports, weather, etc.)
- System or meta-level questions

**Examples:**
- What is the interest rate on Brillar Bank home loans?
- Who is the CEO of Brillar Bank?
- How does inflation work?
- Write a Python script to sort numbers.

---

### 2. RELATED

Use this label if the query is **clearly about Brillar Bank fixed deposits** and can be answered with a **direct lookup or minimal reasoning**.

This includes:
- Single-value lookups
- Simple comparisons
- Basic product details

**Examples:**
- What is the interest rate for a 1-year fixed deposit at Brillar Bank?
- What is the minimum amount to open a fixed deposit?
- Which Brillar Bank FD has the highest interest rate?
- What tenures are available for Brillar Bank fixed deposits?
- I have $5,000,000. What is the best FD strategy using Brillar Bank products to maximize returns with yearly liquidity?
- Compare FD options for a senior citizen investing for 3–5 years.
- If I invest in multiple Brillar Bank FDs, what will be the total maturity amount?
- Design a fixed deposit ladder using Brillar Bank options.
"""
classifier_prompt = """You are a query classifier for a question-answering system that uses **Brillar Bank fixed deposit data** as its main source.

Your task is to read a user query and classify it into **one** of the following categories:

- **NOT_RELATED**  
- **RELATED**

---

## Classification Labels

### 1. NOT_RELATED

Use this label if the query **cannot be answered using Brillar Bank fixed deposit data**.

This includes:
- Questions about other banking products (loans, savings accounts, credit cards, etc.)
- Questions about other banks or general finance topics
- Completely unrelated topics (technology, sports, weather, etc.)
- System or meta-level questions

---

### 2. RELATED

Use this label if the query is **clearly about Brillar Bank fixed deposits** and can be answered with a **direct lookup or minimal reasoning**.

This includes:
- Single-value lookups
- Simple comparisons
- Basic product details
"""

# gpt5 = ChatOpenAI(model="gpt-5-mini", temperature=1, reasoning_effort="minimal")
gpt_oss = ChatOllama(model="gpt-oss:20b", validate_model_on_init=True)
structured_llm = gpt_oss.with_structured_output(ClassifierSchema)


classifier_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", classifier_prompt),
        ("human", "user_query: {question}"),
    ]
)
classifier_chain = classifier_prompt_template | structured_llm

Create QA Chain

In [95]:
instruction = "Answer the questions using the given context only"

qa_instructions = instruction + """\n\n{context}."""
qa_prompt = ChatPromptTemplate.from_messages(
  [("system", qa_instructions), ("human", "{question}")]
)

def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)

formatted_prompt = {
    "question": itemgetter("question") | RunnablePassthrough(),
    "context": lambda x: format_docs(x["context"]),
} | RunnableParallel(prompt=qa_prompt, question=itemgetter("question"))

qa_chain = formatted_prompt | RunnableParallel(
    llm_result=itemgetter("prompt") | llm | StrOutputParser(),
    question=itemgetter("question"),
    )

Adaptive Chain

In [96]:
class AdaptiveSchema(BaseModel):
    """ Schema for Adaptive Loop
    Args:
        final_answer: bool indicating if the final answer is reached
        followup_query: A focused semantic retrieval query for the vector store if final_answer is false
        answer: Answer based on retrieved information.
    """
    final_answer: bool = Field(..., description="Indicates if the final answer is reached")
    followup_question: str = Field(
        ..., description="A follow-up question if final_answer is false"
    )
    answer: str = Field(..., description="Answer based on retrieved information.")


qa_loop_prompt = """You are an analysis agent in an iterative adaptive RAG system.

You **do not** ask the user questions.

When information is missing, you generate **semantic retrieval queries for a vector store**, not clarification questions for a human.

---

## Inputs

You will be given:

1. **One original query**
2. **Zero or more retrieval-query–answer pairs** from previous loops

Each retrieval query is a semantic search request to the same vector store.

---

## Core Principle (VERY IMPORTANT)

✅ **If the available information already answers the original query in a clear, direct, and sufficient way, you MUST consider the query answered.**

❌ **You must NOT:**
- Hedge or speculate
- Introduce uncertainty after the query is answered
- Re-query information that is already retrieved
- Include already-answered entities in follow-up queries

🚨 **Follow-up queries must target ONLY missing information.**

---

## Objective

Your goal is to:

1. **Answer the original query directly and completely using retrieved information**
2. Decide whether the answer **fully satisfies the query as asked**
3. If not satisfied, generate a follow-up retrieval query that retrieves **only the missing parts**

---

## Instructions

1. Read the original query carefully and identify all required parts (e.g., entities, products, attributes).
2. Review retrieved answers and determine which parts are **already answered**.
3. Identify which parts are **still missing**.
4. Ask yourself:

   **“Is every required part of the original query answered?”**

5. If YES → answer confidently and stop.
6. If NO →  
   - Answer using all already-available information  
   - Generate **one follow-up vector retrieval query** that targets **only the missing part(s)**

Do **not** include already-answered information in the follow-up query.
DO NOT mark as fianal_answer true if any part is missing from previous answers.
DO NOT explain reasoning.

---

## Response Logic

### Case 1: The query IS satisfied

Use this case if all required parts of the query are already answered.

- State `"final_answer": true`
- `"followup_query"` must be empty
- `"answer"` must be **direct, decisive, and final**
- Do NOT mention uncertainty or missing context

Output:

{{
  "final_answer": true,
  "followup_query": "",
  "answer": "Direct and complete answer to the original query."
}}

---

### Case 2: The query is NOT satisfied

Use this only if **at least one required part** of the original query is missing.

- State `"final_answer": false`
- `"answer"` should include **all information already retrieved**
- `"followup_query"` must retrieve **only missing information**
- Do NOT include entities, products, or attributes that are already answered

Output:

{{
  "final_answer": false,
  "followup_query": "Semantic retrieval query targeting ONLY missing information",
  "answer": "Best possible answer using all currently retrieved information."
}}

---

## Explicit Examples (Important)

---

### Example A: Multi-Product Comparison (Partial Retrieval)

**Original Query:**  
“Compare Product A, Product B, Product C, Product D, and Product E.”

**Retrieved Information:**  
- Product A details only

✅ Correct output:

{{
  "final_answer": false,
  "followup_query": "Product B, Product C, Product D, and Product E specifications and features",
  "answer": "Product A offers the features and specifications described in the retrieved information."
}}

❌ Incorrect (NOT allowed):
- Follow-up query including Product A
- “Compare all five products” (too broad)

---

### Example B: Attribute-Based Decomposition

**Original Query:**  
“Compare interest rates and withdrawal penalties for Products A and B.”

**Retrieved Information:**  
- Interest rates for Products A and B

✅ Correct output:

{{
  "final_answer": false,
  "followup_query": "Withdrawal penalties for Product A and Product B",
  "answer": "Products A and B have the listed interest rates."
}}

❌ Incorrect:
- Re-querying interest rates
- Asking for full product details again

---

### Example C: Fully Answered Query

**Original Query:**  
“How many types of data are there?”

**Retrieved Information:**  
“There are 5 types of data.”

✅ Correct output:

{{
  "final_answer": true,
  "followup_query": "",
  "answer": "There are five types of data."
}}

---

## Rules

- Follow-up queries MUST retrieve **only missing information**
- Follow-up queries must be **short, keyword-based, and efficient**
- Never re-query answered entities or attributes
- Treat the original query **literally**
- Do NOT speculate or hedge
- Never hallucinate facts
- Generate **at most one** follow-up retrieval query
- Output **only** the JSON-compatible object
- No reasoning, no explanation

---

## Goal

Efficiently answer complex queries by **iteratively retrieving only missing information**, avoiding redundant vector searches, and answering decisively once the query is satisfied."""

qa_loop_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", qa_loop_prompt),
        ("human", "Original Query: \n\n {question} \n Original Answer: \n\n {answer} \n Intermediate Query-Answer Pairs: \n\n {additional_qa} \n "),
    ]
)

gpt5 = ChatOpenAI(model="gpt-5-mini", temperature=1, reasoning_effort="minimal")
# gpt_oss = ChatOllama(model="gpt-oss:20b", validate_model_on_init=True)
structured_llm = gpt5.with_structured_output(AdaptiveSchema)
adaptive_qa_chain = qa_loop_prompt_template | structured_llm

Retriever Chain

In [97]:
retrieve_docs_chain = itemgetter("question") | retriever

QA Loop

In [98]:
def adaptive_loop(_input: dict):
    original_question = _input["question"]
    context = _input["context"]
    original_answer = _input["answer"]["llm_result"]
    additional_qa = []
    all_contexts = context
    k = 0
    print("\n**Original Question:**", original_question)
    # print("**Original Answer:**", original_answer)
    while True:
        loop_input = {
            "question": original_question,
            "answer": original_answer,
            "additional_qa": "\n".join(
                [f"Q: {q}\nA: {a}" for q, a in additional_qa]
            ),
        }
        qa_loop_result = adaptive_qa_chain.invoke(loop_input)
        if qa_loop_result.final_answer or k > 2:
            print("\n**Final Answer:**", original_answer if k == 0 else qa_loop_result.answer)
            return {"answer": qa_loop_result.answer, "context": list(all_contexts)}
        else:
            print("\nIteration:", k+1)
            print("\n**Intermediate Answer:**", qa_loop_result.answer)
            print("\n**Follow-up query:**", qa_loop_result.followup_question)
            followup_question = qa_loop_result.followup_question
            followup_context = retrieve_docs_chain.invoke({"question": followup_question})
            all_contexts.extend(followup_context)
            followup_answer = qa_chain.invoke({
                "question": followup_question,
                "context": followup_context
            })
            additional_qa.append((followup_question, followup_answer)) 
        k += 1

Create Final Chain (Contextualize -> Retrieval -> QA -> Adaptive)

In [99]:
final_qa_chain = RunnablePassthrough.assign(question=itemgetter("question")).assign(context=retrieve_docs_chain).assign(answer=qa_chain).assign(adaptive_answer=lambda x: adaptive_loop(x))


In [100]:
external_answer_prompt = """Answer the user's query using your knowledge."""
external_answer_prompt_template = ChatPromptTemplate.from_messages(
      [
          ("system", external_answer_prompt),
          ("human", "user_query:{question}"),
      ]
  )
llm = ChatOpenAI(model="gpt-5-mini", temperature=1, reasoning_effort="minimal")

external_knowledge_chain = external_answer_prompt_template | llm | StrOutputParser()

In [101]:
@chain
def full_adaptive_chain(_input: dict) -> Runnable:
    print("Classifying the query...", _input["question"])
    result = classifier_chain.invoke({"question": _input["question"]})
    if result.classification == "RELATED":
        print ("**Using internal knowledge for RELATED query.**")
        return final_qa_chain
    else:
        print ("**Using external knowledge for NOT_RELATED query.**")
        return external_knowledge_chain

Invoke Chain

In [102]:
input = "How many types of fixed deposits does Brillar Bank offer?"
final_final_chain = {"question": contextualize_if_needed} | full_adaptive_chain
result = final_final_chain.invoke({"question": input, "chat_history": []})

Classifying the query... How many types of fixed deposits does Brillar Bank offer?
**Using internal knowledge for RELATED query.**

**Original Question:** How many types of fixed deposits does Brillar Bank offer?

**Final Answer:** Brillar Bank offers four types of fixed deposits:
- Fixed Deposit
- Flexi Fixed Deposit
- E-Fixed Deposit
- (Implied) Flexi Fixed Deposit with associated Flexi One Account—treated as part of the Flexi Fixed Deposit offering.


In [107]:
input = "Show me the interest rates for e-fixed deposit, flexi fixed deposit, general fixed deposit, and senior flexi fixed deposit at Brillar Bank?"
final_final_chain = {"question": contextualize_if_needed} | full_adaptive_chain
result = final_final_chain.invoke({"question": input, "chat_history": []})

Classifying the query... Show me the interest rates for e-fixed deposit, flexi fixed deposit, general fixed deposit, and senior flexi fixed deposit at Brillar Bank?
**Using internal knowledge for RELATED query.**

**Original Question:** Show me the interest rates for e-fixed deposit, flexi fixed deposit, general fixed deposit, and senior flexi fixed deposit at Brillar Bank?

Iteration: 1

**Intermediate Answer:** Available information provided earlier lists interest rates for Brillar Bank products as follows:

- E-Fixed Deposit (eFD): 1 month 1.85%; 2–3 months 2.25%; 4–6 months 2.30%; 7–12 months 2.35%; 13–60 months 2.35%.

- Senior Savers Flexi Fixed Deposit: 1 month 2.15%; 2–3 months 2.25%; 4–6 months 2.30%; 7–11 months 2.35%; 12–60 months 2.50%.

- General Fixed Deposit (non-senior) / Fixed Deposit (general): 1 month 2.15%; 2–3 months 2.25%; 4–6 months 2.30%; 7–12 months 2.35%; 13–60 months 2.35%.

These are the rates already provided.

**Follow-up query:** Brillar Bank e-fixed depo

In [ ]:
input = "Tell me the eligibility criteria for Brillar Bank's e-fixed deposit, senior fixed deposit, general fixed deposit, and junior fixed deposit?"
final_final_chain = {"question": contextualize_if_needed} | full_adaptive_chain
result = final_final_chain.invoke({"question": input, "chat_history": []})

Classifying the query... Tell me the eligibility criteria for Brillar Bank's e-fixed deposit, senior flexi fixed deposit, general fixed deposit, and junior fixed deposit?
**Using internal knowledge for RELATED query.**

**Original Question:** Tell me the eligibility criteria for Brillar Bank's e-fixed deposit, senior flexi fixed deposit, general fixed deposit, and junior fixed deposit?

Iteration: 1

**Intermediate Answer:** Available retrieved information summarizes Brillar Bank fixed-deposit products as follows:

- General Fixed Deposit
  - Eligible: Malaysians aged 18 and above.
  - For Malaysians below 18, account must be opened as a trust account.
  - Minimum deposit: RM5,000 for 1-month placement; RM500 for 2 months and above.
  - Product available to individuals and non-individuals.

- Senior Savers Flexi Fixed Deposit
  - Eligible: Malaysians aged 50 and above (single or joint names).
  - Minimum opening deposit: RM10,000.
  - Interest can be earned and withdrawn; partial withd

In [106]:
input = "I have $5,000,000. Out of e-fixed deposit, flexi fixed deposit, general fixed deposit, and senior flexi fixed deposit. which one should I choose to maximize my returns in 1 year and how much will I earn for each?"
final_final_chain = {"question": contextualize_if_needed} | full_adaptive_chain
result = final_final_chain.invoke({"question": input, "chat_history": []})

Classifying the query... I have $5,000,000. Out of e-fixed deposit, flexi fixed deposit, general fixed deposit, and senior flexi fixed deposit. which one should I choose to maximize my returns in 1 year and how much will I earn for each?
**Using internal knowledge for RELATED query.**

**Original Question:** I have $5,000,000. Out of e-fixed deposit, flexi fixed deposit, general fixed deposit, and senior flexi fixed deposit. which one should I choose to maximize my returns in 1 year and how much will I earn for each?

Iteration: 1

**Intermediate Answer:** From the provided information: - Senior Savers Flexi Fixed Deposit (age 50+): 12-month rate = 2.50% p.a. - Flexi Fixed Deposit: 12-month rate = 2.50% p.a. - E-fixed Deposit: 12-month rate not provided in retrieved data - General Fixed Deposit: 12-month rate not provided in retrieved data Based on the two known rates (2.50%): - Earnings on RM5,000,000 at 2.50% for 12 months = RM125,000 Recommendation to maximize 1-year return (given c

In [105]:
input = "How many types of fixed deposit product Brillar Bank and Hong Leong Bank have in common?"
result = final_final_chain.invoke(
        {"question": input, "chat_history": []},
    )


Classifying the query... How many types of fixed deposit product Brillar Bank and Hong Leong Bank have in common?
**Using external knowledge for NOT_RELATED query.**
